# Unit Testing ML Code: Hands-on Exercise (Data Engineering)

## In this notebook we will explore unit tests for data engineering

#### We will use a classic toy dataset: the Iris plants dataset, which comes included with scikit-learn
Dataset details: https://scikit-learn.org/stable/datasets/index.html#iris-plants-dataset

As we progress through the course, the complexity of examples will increase, but we will start with something basic. This notebook is designed so that it can be run in isolation, once the setup steps described below are complete.

### Setup

Let's begin by importing the dataset and the libraries we are going to use. Make sure you have run `pip install -r requirements.txt` on requirements file located in the same directory as this notebook. We recommend doing this in a separate virtual environment (see dedicated setup lecture).

If you need a refresher on jupyter, pandas or numpy, there are some links to resources in the section notes.

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

# Access the iris dataset from sklearn
iris = datasets.load_iris()

# Load the iris data into a pandas dataframe. The `data` and `feature_names`
# attributes of the dataset are added by default by sklearn. We use them to
# specify the columns of our dataframes.
iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)

# Create a "target" column in our dataframe, and set the values to the correct
# classifications from the dataset.
iris_frame['target'] = iris.target

In [32]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

### Add the `SimplePipeline` from the Test Input Values notebook (same as previous lecture, no changes here)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        # Shorthand to specify that each value should start out as
        # None when the class is instantiated.
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Load the dataset and perform train test split."""
        # fetch from sklearn
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        for col in self.frame.columns:
            self.frame[col] *= -1
        self.frame['target'] = dataset.target
        
        
        # we divide the data set using the train_test_split function from sklearn, 
        # which takes as parameters, the dataframe with the predictor variables, 
        # then the target, then the percentage of data to assign to the test set, 
        # and finally the random_state to ensure reproducibility.
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        # we set up a LogisticRegression classifier with default parameters
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        
        # use our X_test and y_test values generated when we used
        # `train_test_split` to test accuracy.
        # score is a method on the Logisitic Regression that 
        # returns the accuracy by default, but can be changed to other metrics, see: 
        # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.train()

### Test Engineered Data (preprocessing)

Below we create an updated pipeline which inherits from the SimplePipeline but has new functionality to preprocess the data by applying a scaler. Linear models are sensitive to the scale of the features. For example features with bigger magnitudes tend to dominate if we do not apply a scaler.

In [34]:
from sklearn.preprocessing import StandardScaler


class PipelineWithDataEngineering(SimplePipeline):
    def __init__(self):
        # Call the inherited SimplePipeline __init__ method first.
        super().__init__()
        
        # scaler to standardize the variables in the dataset
        self.scaler = StandardScaler()
        # Train the scaler once upon pipeline instantiation:
        # Compute the mean and standard deviation based on the training data
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        # Scale the test and training data to be of mean 0 and of unit variance
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        # apply scaler transform on inputs before predictions
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.apply_scaler()  # updated in the this class
        self.train()

In [35]:
pipeline = PipelineWithDataEngineering()
pipeline.run_pipeline()
accuracy_score = pipeline.get_accuracy()
print(f'current model accuracy is: {accuracy_score}')

current model accuracy is: 0.9591836734693877


### Now we Unit Test
We focus specifically on the feature engineering step

In [36]:
pipeline.load_dataset()
# pd.DataFrame(pipeline.X_train).stack().mean()

In [30]:
for col in pipeline.X_train.columns:
    pipeline.X_train[col] *= -1

In [37]:
pipeline.X_train

,sepal length,sepal width,petal length,petal width
119,-6.0,-2.2,-5.0,-1.5
125,-7.2,-3.2,-6.0,-1.8
3,-4.6,-3.1,-1.5,-0.2
17,-5.1,-3.5,-1.4,-0.3
38,-4.4,-3.0,-1.3,-0.2
72,-6.3,-2.5,-4.9,-1.5
136,-6.3,-3.4,-5.6,-2.4
6,-4.6,-3.4,-1.4,-0.3
112,-6.8,-3.0,-5.5,-2.1
100,-6.3,-3.3,-6.0,-2.5


In [38]:
import unittest


class TestIrisDataEngineering(unittest.TestCase):

    def setUp(self):
        """Call the first method of the tested class after instantiating"""
        self.pipeline = PipelineWithDataEngineering()
        self.pipeline.load_dataset()
    
    def test_scaler_preprocessing_brings_x_train_mean_near_zero(self):
        """"""
        # Given
        # convert the dataframe to be a single column with pandas stack
        original_mean = self.pipeline.X_train.stack().mean()
        
        # When
        self.pipeline.apply_scaler()
        
        # Then
        # The idea behind StandardScaler is that it will transform your data 
        # to center the distribution at 0 and scale the variance at 1.
        # Therefore we test that the mean has shifted to be less than the original
        # and close to 0 using assertAlmostEqual to check to 3 decimal places:
        # https://docs.python.org/3/library/unittest.html#unittest.TestCase.assertAlmostEqual
        self.assertTrue(original_mean > self.pipeline.X_train.mean())  # X_train is a numpy array at this point.
        self.assertAlmostEqual(self.pipeline.X_train.mean(), 0.0, places=3)
        print(f'Original X train mean: {original_mean}')
        print(f'Transformed X train mean: {self.pipeline.X_train.mean()}')
        
    def test_scaler_preprocessing_brings_x_train_std_near_one(self):
        # When
        self.pipeline.apply_scaler()
        
        # Then
        # We also check that the standard deviation is close to 1
        self.assertAlmostEqual(self.pipeline.X_train.std(), 1.0, places=3)
        print(f'Transformed X train standard deviation : {self.pipeline.X_train.std()}')
        

In [39]:
import sys
suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisDataEngineering)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

F.

Transformed X train standard deviation : 1.0



FAIL: test_scaler_preprocessing_brings_x_train_mean_near_zero (__main__.TestIrisDataEngineering)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-38-5f655c934106>", line 26, in test_scaler_preprocessing_brings_x_train_mean_near_zero
    self.assertTrue(original_mean > self.pipeline.X_train.mean())  # X_train is a numpy array at this point.
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.048s

FAILED (failures=1)


<unittest.runner.TextTestResult run=2 errors=0 failures=1>

## Data Engineering Test: Hands-on Exercise
Change the pipeline class preprocessing so that the test fails. Do you understand why the test is failing?